In [ ]:
from pose_graph import PoseGraph
import numpy as np

In [ ]:
sensor_id = "sensor_1"
print( "The sensor "+ sensor_id + " is already in the graph \n please use update_graph() to update the edges")

In [ ]:
sensor_id = ["cam0", "cam1", "imu0", "imu1"]

pose_id = ["cam1", "cam2", "cam3", "cam4", "cam5"]

poses = {
    "cam1": [[1, "M"], [2, "N"], [3, "O"], [4, "P"], [5, "Q"]],
    "cam2": [(2, "N")],
    "cam3": [(3, "O")],
    "cam4": [(4, "P")],
    "cam5": [(5, "Q")],
}

# for pose in poses.items():
#     print(pose[1][1])

if 1 not in poses["cam1"]:
    print("not in")

print(poses["cam1"])


In [ ]:
edge = {"sensor": {"cam0": 1, "cam1": 2, "imu0": 3, "imu1": 4}}

# if "sensor" in edge:
#     print("yes")

# else:
#     print("no") 

# try:
#     edge["sensor"]["cam2"]
#     print("yes")
# except KeyError:
#     print("no")

if "cam0" in edge.get("sensor",{}):
    print("yes")


str1 = "cam0"

list1 = ["cam0", "cam1", "cam2", "cam3", "cam4", "cam5"]
newstr = "cam6"+str1
list1.append("cam6"+str1)
print(list1)

In [ ]:
list1 = ["cam0", "cam1", "cam2", "cam3", "cam4", "cam5"]
for item in list1:
    string1 = "cam6"+item
    print(string1)

In [1]:
from pose_graph import PoseGraph
import numpy as np
pg = PoseGraph()

sensor_id = "0"

tf00 = np.array(
    [
        [1, 0, 0, 1],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf01 = np.array(
    [
        [1, 0, 0, 0],
        [0, 0, -1, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf02 = np.array(
    [
        [1, 0, 0, 0],
        [0, 0, -1, 2],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

pg.add_sensor(
    sensor_id,
    {
        "0": [tf00, True],
        "1": [tf01, True],
        "2": [tf02, True],
    },
)

sensor_id = "1"

tf10 = np.array(
    [
        [1, 0, 0, 0],
        [0, 0, -1, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf11 = np.array(
    [
        [1, 0, 0, 1],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf12 = np.array(
    [
        [1, 0, 0, 1],
        [0, 0, -1, 2],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

pg.add_sensor(
    sensor_id,
    {
        "0": [tf10, True],
        "1": [tf11, True],
        "2": [tf12, True],
    },
)

pg.update_graph(
    "0",
    {
        "0": [tf00, False],
        "1": [tf01, True],
        "2": [tf02, True],
    },
)

tf00 = pg.get_transform(parent_id="0", child_id="0", solver_method="BFS")

print(tf00)

# print(pg.object_id)
# print(pg.edges)

Node id sensor_0
Path ['sensor_0']
The child node is object_1
The child node is object_2
Node id object_1
Path ['sensor_0', 'object_1']
The child node is sensor_0
The child node is sensor_1
Node id object_2
Path ['sensor_0', 'object_2']
The child node is sensor_0
The child node is sensor_1
Node id sensor_1
Path ['sensor_0', 'object_1', 'sensor_1']
The child node is object_0
The child node is object_1
The child node is object_2
Node id object_0
Path ['sensor_0', 'object_1', 'sensor_1', 'object_0']
The path between the parent node and the child node is:  ['sensor_0', 'object_1', 'sensor_1', 'object_0']
[[ 1.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]


In [ ]:
tf01 = np.array([
    [1, 0, 0, 0],
    [0, 0, -1, 1],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
], dtype=np.float32)

tf11 = np.array([
    [1, 0, 0, 1],
    [0, 0, -1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
], dtype=np.float32)

tf10 = np.array([
    [1, 0, 0, 0],
    [0, 0, -1, 1],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
], dtype=np.float32)

tf00 = tf01 @ np.linalg.inv(tf11) @ tf10

print(tf00)